In [1]:
import pandas as pd
import numpy as np

THE_MOST_IMPORTANT_K = 16 #Changed this, so i can use it in the split function as well, without any kind of refactoring.

In [2]:
names = (
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'income',
)

In [3]:
categorical = [
    'workclass',
    #'education', THIS COLUMN DELETED FROM HERE SINCE WE DROP IT ANYWAY
    'marital-status',
    'occupation',
    'relationship',
    'sex',
    'native-country',
    'race',
    'income',
]
df = pd.read_csv("adult.data", sep=", ", header=None, names=names, index_col=False, engine='python');# We load the data using Pandas

In [4]:
df = df.drop(columns=['fnlwgt','education'])

In [5]:
for name in categorical:
    df[name] = df[name].astype('category')

In [6]:
def get_spans(df, partition, scale=None):
    """
    :param        df: the dataframe for which to calculate the spans
    :param partition: the partition for which to calculate the spans
    :param     scale: if given, the spans of each column will be divided
                      by the value in `scale` for that column
    :        returns: The spans of all columns in the partition
    """
    spans = {}
    for column in df.columns:
        if column in categorical:
            span = len(df[column][partition].unique())
        else:
            span = df[column][partition].max()-df[column][partition].min()
        if scale is not None:
            span = span/scale[column]
        spans[column] = span
    return spans

In [7]:
full_spans = get_spans(df, df.index)

In [8]:
def split(df, partition, column):
    """
    :param        df: The dataframe to split
    :param partition: The partition to split
    :param    column: The column along which to split
    :        returns: A tuple containing a split of the original partition
    """
    dfp = df[column][partition]
    if column in categorical:
        # Commenting out the next 4 lines should make the world a better place, but who am i to know this for sure...
        # But it does not... Reality is often disappointing.... -_- 
        values = dfp.unique()
        lv = set(values[:len(values)//2])
        rv = set(values[len(values)//2:])
        return dfp.index[dfp.isin(lv)], dfp.index[dfp.isin(rv)]
        
        indexes = dfp.value_counts().index
        values = dfp.value_counts().values
        
        zipped = zip(indexes,values)
        sortedvals = np.array(sorted(zipped, key=lambda x: x[1]))
        sortedvalues = np.array(sortedvals[:,1],dtype=int)
        cumulated = np.cumsum(sortedvalues)
        
        bestindex = 0
        for i in range(len(cumulated)):
            if cumulated[i]>=THE_MOST_IMPORTANT_K and (cumulated[-1]-cumulated[i])>=THE_MOST_IMPORTANT_K and cumulated[i]<cumulated[-1]/2:
                bestindex = i
        if cumulated[bestindex+1]>=THE_MOST_IMPORTANT_K and (cumulated[-1]-cumulated[bestindex+1])>=THE_MOST_IMPORTANT_K:
            if np.abs(cumulated[-1]/2-cumulated[bestindex])>np.abs(cumulated[-1]/2-cumulated[bestindex+1]):
                bestindex+=1
        
        lv = set(sortedvals[:bestindex,0])
        rv = set(sortedvals[bestindex:,0])
        return dfp.index[dfp.isin(lv)], dfp.index[dfp.isin(rv)]
    else:        
        median = dfp.median()
        dfl = dfp.index[dfp < median]
        dfr = dfp.index[dfp >= median]
        
        if len(dfl) < THE_MOST_IMPORTANT_K or len(dfr) < THE_MOST_IMPORTANT_K:
            median = dfp.mean() #Very clever hack, 200IQ code... xD
        
        dfl = dfp.index[dfp < median]
        dfr = dfp.index[dfp >= median]
        
        return (dfl, dfr)

In [9]:
def is_k_anonymous(df, partition, sensitive_column):
    """
    :param               df: The dataframe on which to check the partition.
    :param        partition: The partition of the dataframe to check.
    :param sensitive_column: The name of the sensitive column
    :param                k: The desired k
    :returns               : True if the partition is valid according to our k-anonymity criteria, False otherwise.
    """
    #COMMENT OUT THE NEXT 4 LINES IF WE DO NOT WANT TO ENFORCE DIFFERENT SENSITIVE COLUMNS IN PARTITIONS
    
    #sensitive_counts = df.loc[partition].groupby(sensitive_column).agg({sensitive_column : 'count'})
    #for sensitive_value, count in sensitive_counts[sensitive_column].items():
        #if count == 0:
            #return False
    
    if len(partition) < THE_MOST_IMPORTANT_K:
        return False
    return True

def partition_dataset(df, feature_columns, sensitive_column, scale, is_valid):
    """
    :param               df: The dataframe to be partitioned.
    :param  feature_columns: A list of column names along which to partition the dataset.
    :param sensitive_column: The name of the sensitive column (to be passed on to the `is_valid` function)
    :param            scale: The column spans as generated before.
    :param         is_valid: A function that takes a dataframe and a partition and returns True if the partition is valid.
    :returns               : A list of valid partitions that cover the entire dataframe.
    """
    finished_partitions = []
    partitions = [df.index]
    while partitions:
        partition = partitions.pop(0)
        #Lil' faster
        if len(partition)<2*THE_MOST_IMPORTANT_K:
            finished_partitions.append(partition)
            continue
        spans = get_spans(df[feature_columns], partition, scale)
        for column, span in sorted(spans.items(), key=lambda x:-x[1]):
            lp, rp = split(df, partition, column)
            if not is_valid(df, lp, sensitive_column) or not is_valid(df, rp, sensitive_column):
                continue
            partitions.extend((lp, rp))
            break
        else:
            finished_partitions.append(partition)
    return finished_partitions

In [10]:
feature_columns = ['age', 'education-num','race','native-country','workclass']
sensitive_column = 'income'
finished_partitions = partition_dataset(df, feature_columns, sensitive_column, full_spans, is_k_anonymous)

In [11]:
# we get the number of partitions that were created
len(finished_partitions)

938

In [12]:
def agg_categorical_column(series):
    return [','.join(set(series))]

def agg_numerical_column(series):
    return [series.mean()]

In [13]:
def build_anonymized_dataset_2(df, partitions, feature_columns, sensitive_column, max_partitions=None):
    aggregations = {}
    for column in feature_columns:
        if column in categorical:
            aggregations[column] = agg_categorical_column
        else:
            aggregations[column] = agg_numerical_column
    rows = []
    for i, partition in enumerate(partitions):
        if i % 100 == 1:
            print("Finished {} partitions...".format(i))
        if max_partitions is not None and i > max_partitions:
            break
        grouped_columns = df.loc[partition].agg(aggregations, squeeze=False)
        sensitive_counts = df.loc[partition].groupby(sensitive_column).agg({sensitive_column : 'count'})
        values = grouped_columns.iloc[0].to_dict()
        IsOkay = True
        #COMMENT OUT THE NEXT 3 LINES IF WE DO NOT WANT TO ENFORCE DIFFERENT SENSITIVE COLUMNS IN PARTITIONS
        #for sensitive_value, count in sensitive_counts[sensitive_column].items():
            #if count == 0:
                #IsOkay=False
        if IsOkay:
            for rowind in range(len(df.loc[partition])):
                currow = df.loc[partition].iloc[rowind,:].copy()
                for feature in feature_columns:
                    currow[feature]=grouped_columns[feature][0]
                rows.append(currow.copy())
    return pd.DataFrame(rows)

In [14]:
dfn2 = build_anonymized_dataset_2(df, finished_partitions, feature_columns, sensitive_column)

Finished 1 partitions...
Finished 101 partitions...
Finished 201 partitions...
Finished 301 partitions...
Finished 401 partitions...
Finished 501 partitions...
Finished 601 partitions...
Finished 701 partitions...
Finished 801 partitions...
Finished 901 partitions...


In [15]:
dfn2

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
1864,49.83871,"State-gov,Without-pay,Federal-gov,Local-gov",8.516129,Married-civ-spouse,Protective-serv,Husband,"Amer-Indian-Eskimo,Asian-Pac-Islander,Other",Male,0,0,54,"Philippines,?,China,United-States",>50K
2526,49.83871,"State-gov,Without-pay,Federal-gov,Local-gov",8.516129,Divorced,Prof-specialty,Unmarried,"Amer-Indian-Eskimo,Asian-Pac-Islander,Other",Female,0,0,40,"Philippines,?,China,United-States",<=50K
2601,49.83871,"State-gov,Without-pay,Federal-gov,Local-gov",8.516129,Married-civ-spouse,Transport-moving,Husband,"Amer-Indian-Eskimo,Asian-Pac-Islander,Other",Male,0,0,40,"Philippines,?,China,United-States",<=50K
5663,49.83871,"State-gov,Without-pay,Federal-gov,Local-gov",8.516129,Divorced,Craft-repair,Not-in-family,"Amer-Indian-Eskimo,Asian-Pac-Islander,Other",Male,0,0,40,"Philippines,?,China,United-States",>50K
6184,49.83871,"State-gov,Without-pay,Federal-gov,Local-gov",8.516129,Married-civ-spouse,Protective-serv,Husband,"Amer-Indian-Eskimo,Asian-Pac-Islander,Other",Male,0,0,40,"Philippines,?,China,United-States",>50K
7375,49.83871,"State-gov,Without-pay,Federal-gov,Local-gov",8.516129,Never-married,Protective-serv,Own-child,"Amer-Indian-Eskimo,Asian-Pac-Islander,Other",Male,4650,0,48,"Philippines,?,China,United-States",<=50K
9015,49.83871,"State-gov,Without-pay,Federal-gov,Local-gov",8.516129,Married-civ-spouse,Other-service,Husband,"Amer-Indian-Eskimo,Asian-Pac-Islander,Other",Male,0,0,40,"Philippines,?,China,United-States",<=50K
9266,49.83871,"State-gov,Without-pay,Federal-gov,Local-gov",8.516129,Married-civ-spouse,Exec-managerial,Husband,"Amer-Indian-Eskimo,Asian-Pac-Islander,Other",Male,0,0,48,"Philippines,?,China,United-States",<=50K
9568,49.83871,"State-gov,Without-pay,Federal-gov,Local-gov",8.516129,Divorced,Other-service,Not-in-family,"Amer-Indian-Eskimo,Asian-Pac-Islander,Other",Female,0,0,40,"Philippines,?,China,United-States",<=50K
10480,49.83871,"State-gov,Without-pay,Federal-gov,Local-gov",8.516129,Married-civ-spouse,Other-service,Husband,"Amer-Indian-Eskimo,Asian-Pac-Islander,Other",Male,0,0,40,"Philippines,?,China,United-States",<=50K


In [16]:
import matplotlib.pyplot as plt
import numpy as np

sizes = np.zeros(len(finished_partitions))
for i in range(len(finished_partitions)):
    sizes[i]=finished_partitions[i].shape[0]

plt.figure()
plt.hist(sizes)
plt.savefig(str(THE_MOST_IMPORTANT_K)+"_census.png")

In [17]:
#ONE HOT encoding the whole thing

uniqitems = {}
for col in categorical:
    uniqitems[col]=df[col].unique()

rows = []

for rowind in range(len(dfn2)):
    if rowind %500 ==0:
        print(rowind/len(dfn2)*100)
    currow = dfn2.iloc[rowind,:].copy()
    for col in categorical:
        if col != 'income':
            values = currow[col].split(',')
            
            for possibleitem in uniqitems[col]:
                if possibleitem in values:
                    currow=currow.append(pd.Series([1/len(values)],[col+'_'+possibleitem]))
                else:
                    currow=currow.append(pd.Series([0],[col+'_'+possibleitem]))
            currow=currow.drop(col)
    rows.append(currow.copy())

0.0
1.5355793740978472
3.0711587481956943
4.606738122293541
6.142317496391389
7.677896870489236
9.213476244587081
10.74905561868493
12.284634992782777
13.820214366880624
15.355793740978472
16.89137311507632
18.426952489174163
19.962531863272012
21.49811123736986
23.03369061146771
24.569269985565555
26.1048493596634
27.640428733761247
29.176008107859097
30.711587481956943
32.24716685605479
33.78274623015264
35.31832560425048
36.853904978348325
38.38948435244618
39.925063726544025
41.46064310064187
42.99622247473972
44.53180184883757
46.06738122293542
47.60296059703326
49.13853997113111
50.67411934522895
52.2096987193268
53.74527809342465
55.280857467522495
56.816436841620344
58.352016215718194
59.88759558981604
61.42317496391389
62.95875433801174
64.49433371210958
66.02991308620743
67.56549246030528
69.10107183440311
70.63665120850096
72.17223058259881
73.70780995669665
75.24338933079451
76.77896870489236
78.3145480789902
79.85012745308805
81.3857068271859
82.92128620128373
84.456865575

In [18]:
final_set = pd.DataFrame(rows)

In [19]:
final_set.to_csv(str(THE_MOST_IMPORTANT_K)+"_census.csv")

In [20]:
final_set

,age,education-num,capital-gain,capital-loss,hours-per-week,income,workclass_State-gov,workclass_Self-emp-not-inc,workclass_Private,workclass_Federal-gov,...,native-country_Vietnam,native-country_Hong,native-country_Ireland,native-country_Hungary,native-country_Holand-Netherlands,race_White,race_Black,race_Asian-Pac-Islander,race_Amer-Indian-Eskimo,race_Other
0,49.83871,8.516129,0,0,54,>50K,0.25,0.0,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333
1,49.83871,8.516129,0,0,40,<=50K,0.25,0.0,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333
2,49.83871,8.516129,0,0,40,<=50K,0.25,0.0,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333
3,49.83871,8.516129,0,0,40,>50K,0.25,0.0,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333
4,49.83871,8.516129,0,0,40,>50K,0.25,0.0,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333
5,49.83871,8.516129,4650,0,48,<=50K,0.25,0.0,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333
6,49.83871,8.516129,0,0,40,<=50K,0.25,0.0,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333
7,49.83871,8.516129,0,0,48,<=50K,0.25,0.0,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333
8,49.83871,8.516129,0,0,40,<=50K,0.25,0.0,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333
9,49.83871,8.516129,0,0,40,<=50K,0.25,0.0,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333
